# HMBA Basal Ganglia Dataset: Comprehensive EDA & ML

## Human and Mammalian Brain Atlas (HMBA) Basal Ganglia Dataset

This notebook provides comprehensive exploratory data analysis and machine learning workflows for the HMBA Basal Ganglia single-nucleus RNA sequencing dataset.

**Dataset Highlights:**
- ~1.9 million nuclei across multiple primate species
- 16,630 genes (one-to-one orthologs)
- Hierarchical cell type taxonomy
- Both raw and log2-transformed expression matrices

**This notebook demonstrates:**
1. Loading data via MLCommons Croissant metadata
2. Comprehensive EDA with visualizations
3. ML model training with PyTorch and TensorFlow
4. Model behavior analysis

## 1. Setup & Installation

In [ ]:
# Install required packages
!pip install mlcroissant pandas numpy matplotlib seaborn scikit-learn torch tensorflow -q

import sys
print(f"Python version: {sys.version}")

In [ ]:
import mlcroissant as mlc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print(f"pandas version: {pd.__version__}")

## 2. Load Dataset via FAIR² Metadata

The FAIR² JSON-LD file (based on the MLCommons Croissant standard) provides machine-readable metadata and enables loading data directly from the dataset's DOI-based URL.

In [ ]:
# Load the FAIR² dataset from the DOI-based URL
fair2_url = 'https://sen.science/doi/10.71728/senscience.xe0q-5vu6/fair2.json'
dataset = mlc.Dataset(fair2_url)
metadata = dataset.metadata.to_json()

print(f"Dataset: {metadata['name']}")
print(f"Version: {metadata.get('version', 'N/A')}")
print(f"License: {metadata.get('license', 'N/A')}")
print(f"\nDescription: {metadata.get('description', 'N/A')[:200]}...")

In [ ]:
# List all available files in the dataset
distributions = metadata.get('distribution', [])

print(f"Dataset contains {len(distributions)} files:")
print("="*70)

# Group by type
h5ad_files = [d for d in distributions if 'hdf5' in d.get('encodingFormat', '')]
parquet_files = [d for d in distributions if 'parquet' in d.get('encodingFormat', '')]
csv_files = [d for d in distributions if 'csv' in d.get('encodingFormat', '')]

print(f"\n📦 H5AD Files ({len(h5ad_files)}):")
for f in h5ad_files:
    print(f"   • {f['name']}")

print(f"\n📊 Parquet Files ({len(parquet_files)}):")
for f in parquet_files:
    print(f"   • {f['name']}")

print(f"\n📄 CSV Files ({len(csv_files)}):")
for f in csv_files:
    print(f"   • {f['name']}")

## 3. Load Data via Croissant RecordSets

We'll load the dataset tables through the FAIR² metadata using `dataset.records()`. The RecordSets read from Parquet files hosted at the dataset's DOI URL for significantly faster loading and smaller file sizes.

In [ ]:
# Helper function to load records from Croissant dataset
def load_croissant_records(dataset, record_set_name, max_records=100000):
    """Load records from a Croissant dataset RecordSet into a DataFrame.
    
    Note: mlcroissant returns column names prefixed with the RecordSet ID
    (e.g., 'donor_records/donor_label'). This function strips the prefix
    to return clean column names matching the original CSV headers.
    
    When reading from Parquet files, mlcroissant may return string values
    as bytes objects (e.g., b'Homo sapiens'). This function decodes them
    back to Python str for seamless downstream use.
    """
    try:
        records = list(dataset.records(record_set=record_set_name))
        if max_records and len(records) > max_records:
            records = records[:max_records]
        df = pd.DataFrame(records)
        # Strip RecordSet prefix from column names (e.g., 'donor_records/donor_label' -> 'donor_label')
        df.columns = [col.split('/')[-1] if '/' in col else col for col in df.columns]
        # Decode bytes columns to str (mlcroissant + Parquet returns bytes for string fields)
        for col in df.columns:
            if df[col].dtype == object and len(df) > 0 and isinstance(df[col].iloc[0], bytes):
                df[col] = df[col].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
        print(f"✓ Loaded '{record_set_name}': {len(df):,} rows, {len(df.columns)} columns")
        return df
    except Exception as e:
        print(f"✗ Could not load '{record_set_name}': {e}")
        return None

# List available RecordSets in Croissant metadata
print("="*70)
print("AVAILABLE RECORDSETS IN CROISSANT METADATA")
print("="*70)
record_sets = metadata.get('recordSet', [])
for rs in record_sets:
    print(f"  • {rs['@id']}: {rs['name']}")

print("\n" + "="*70)
print("LOADING DATA FROM CROISSANT RECORDSETS")
print("="*70 + "\n")

# Load metadata tables via Croissant RecordSets (backed by Parquet files)
# Core tables
df_donor = load_croissant_records(dataset, 'donor_records')
df_cluster = load_croissant_records(dataset, 'cluster_records')
df_taxonomy = load_croissant_records(dataset, 'cluster_annotation_term_records')
df_taxonomy_levels = load_croissant_records(dataset, 'cluster_annotation_term_set_records')
df_library = load_croissant_records(dataset, 'library_records')
df_gene = load_croissant_records(dataset, 'gene_csv_records')

# Cell-level data (limited for performance)
df_cell_metadata = load_croissant_records(dataset, 'cell_metadata_csv_records', max_records=100000)
df_cell_cluster = load_croissant_records(dataset, 'cell_to_cluster_membership_records', max_records=100000)
df_embeddings = load_croissant_records(dataset, 'cell_2d_embedding_records', max_records=100000)

# Annotation mappings
df_cluster_to_annotation = load_croissant_records(dataset, 'cluster_to_cluster_annotation_records')
df_abbreviations = load_croissant_records(dataset, 'cluster_annotation_to_abbreviation_records')
df_abbreviation_terms = load_croissant_records(dataset, 'abbreviation_term_records')

# Reference tables
df_value_sets = load_croissant_records(dataset, 'value_sets_records')
df_example_expression = load_croissant_records(dataset, 'example_gene_expression_records')

print("\n" + "="*70)
print("✅ ALL DATA LOADED VIA CROISSANT RECORDSETS (Parquet-backed)")
print("="*70)

# Summary of loaded data
loaded_dfs = {
    'donor': df_donor,
    'cluster': df_cluster,
    'taxonomy (terms)': df_taxonomy,
    'taxonomy (levels)': df_taxonomy_levels,
    'library': df_library,
    'gene': df_gene,
    'cell_metadata': df_cell_metadata,
    'cell_cluster': df_cell_cluster,
    'embeddings': df_embeddings,
    'cluster_to_annotation': df_cluster_to_annotation,
    'abbreviations_map': df_abbreviations,
    'abbreviation_terms': df_abbreviation_terms,
    'value_sets': df_value_sets,
    'example_expression': df_example_expression
}

print("\nLoaded DataFrames Summary:")
for name, df in loaded_dfs.items():
    if df is not None:
        print(f"  {name}: {len(df):,} rows × {len(df.columns)} cols")

## 4. Exploratory Data Analysis

### 4.1 Dataset Overview

In [ ]:
# Dataset summary statistics
print("="*60)
print("HMBA BASAL GANGLIA DATASET OVERVIEW")
print("="*60)

if df_donor is not None:
    print(f"\n🧬 Donors: {len(df_donor)}")
    if 'species_scientific_name' in df_donor.columns:
        species_counts = df_donor['species_scientific_name'].value_counts()
        for species, count in species_counts.items():
            print(f"   • {species}: {count} donors")

if df_cluster is not None:
    print(f"\n🔬 Clusters: {len(df_cluster)}")
    if 'number_of_cells' in df_cluster.columns:
        total_cells = df_cluster['number_of_cells'].sum()
        print(f"   • Total cells: {total_cells:,}")

if df_gene is not None:
    print(f"\n🧪 Genes: {len(df_gene):,}")

if df_library is not None:
    print(f"\n📚 Libraries: {len(df_library)}")

if df_taxonomy is not None:
    print(f"\n📊 Taxonomy Terms: {len(df_taxonomy)}")
    if 'cluster_annotation_term_set_name' in df_taxonomy.columns:
        term_sets = df_taxonomy['cluster_annotation_term_set_name'].unique()
        print(f"   • Hierarchy levels: {len(term_sets)}")
        for ts in term_sets[:5]:
            print(f"     - {ts}")

### 4.2 Donor & Species Distribution

In [ ]:
if df_donor is not None and len(df_donor) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(14, 4))
    fig.suptitle('Donor Demographics', fontsize=14, fontweight='bold', y=1.02)
    
    # Species distribution
    ax1 = axes[0]
    if 'species_scientific_name' in df_donor.columns:
        species_counts = df_donor['species_scientific_name'].value_counts()
        colors = sns.color_palette('Set2', len(species_counts))
        bars = ax1.bar(range(len(species_counts)), species_counts.values, color=colors)
        ax1.set_xticks(range(len(species_counts)))
        ax1.set_xticklabels([s.replace(' ', '\n') for s in species_counts.index], fontsize=9)
        ax1.set_ylabel('Number of Donors')
        ax1.set_title('A. Species Distribution')
        for bar, val in zip(bars, species_counts.values):
            ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                    str(val), ha='center', va='bottom', fontsize=10)
    
    # Sex distribution
    ax2 = axes[1]
    if 'donor_sex' in df_donor.columns:
        sex_counts = df_donor['donor_sex'].value_counts()
        colors = ['#FF6B6B' if 'F' in str(s).upper() else '#4ECDC4' for s in sex_counts.index]
        wedges, texts, autotexts = ax2.pie(sex_counts.values, labels=sex_counts.index, 
                                           autopct='%1.1f%%', colors=colors,
                                           wedgeprops=dict(edgecolor='white', linewidth=2))
        ax2.set_title('B. Sex Distribution')
    
    # Age distribution
    ax3 = axes[2]
    if 'donor_age_value' in df_donor.columns:
        ages = pd.to_numeric(df_donor['donor_age_value'], errors='coerce').dropna()
        if len(ages) > 0:
            ax3.hist(ages, bins=15, color='#95E1D3', edgecolor='white', linewidth=1.2)
            ax3.axvline(ages.mean(), color='#F38181', linestyle='--', linewidth=2, label=f'Mean: {ages.mean():.1f}')
            ax3.set_xlabel('Age')
            ax3.set_ylabel('Count')
            ax3.set_title('C. Age Distribution')
            ax3.legend()
    
    plt.tight_layout()
    plt.savefig('fig1_donor_demographics.png', dpi=150, bbox_inches='tight', facecolor='white')
    plt.show()
else:
    print("Donor data not available for visualization.")

### 4.3 Cluster Distribution & Cell Counts

In [ ]:
if df_cluster is not None and 'number_of_cells' in df_cluster.columns:
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    fig.suptitle('Cluster Analysis', fontsize=14, fontweight='bold', y=1.02)
    
    # Cell count distribution (log scale)
    ax1 = axes[0]
    cell_counts = df_cluster['number_of_cells'].values
    ax1.hist(np.log10(cell_counts + 1), bins=30, color='#6C5CE7', edgecolor='white', alpha=0.8)
    ax1.set_xlabel('log₁₀(Cell Count + 1)')
    ax1.set_ylabel('Number of Clusters')
    ax1.set_title('A. Cell Count Distribution per Cluster')
    ax1.axvline(np.log10(np.median(cell_counts) + 1), color='#FD79A8', linestyle='--', 
                linewidth=2, label=f'Median: {int(np.median(cell_counts)):,}')
    ax1.legend()
    
    # Top 20 clusters by cell count
    ax2 = axes[1]
    top_clusters = df_cluster.nlargest(20, 'number_of_cells')
    if 'label' in top_clusters.columns:
        labels = [l[:20] + '...' if len(str(l)) > 20 else str(l) for l in top_clusters['label']]
    elif 'cluster_alias' in top_clusters.columns:
        labels = [str(l)[:20] for l in top_clusters['cluster_alias']]
    else:
        labels = [f"Cluster {i}" for i in range(len(top_clusters))]
    
    colors = plt.cm.viridis(np.linspace(0.2, 0.8, 20))
    ax2.barh(range(len(top_clusters)), top_clusters['number_of_cells'].values, color=colors[:len(top_clusters)])
    ax2.set_yticks(range(len(top_clusters)))
    ax2.set_yticklabels(labels, fontsize=8)
    ax2.set_xlabel('Number of Cells')
    ax2.set_title('B. Top 20 Clusters by Cell Count')
    ax2.invert_yaxis()
    
    # Cumulative distribution
    ax3 = axes[2]
    sorted_counts = np.sort(cell_counts)[::-1]
    cumsum = np.cumsum(sorted_counts) / sorted_counts.sum() * 100
    ax3.plot(range(1, len(cumsum) + 1), cumsum, color='#00B894', linewidth=2)
    ax3.fill_between(range(1, len(cumsum) + 1), cumsum, alpha=0.3, color='#00B894')
    ax3.axhline(80, color='#FDCB6E', linestyle='--', label='80% threshold')
    idx_80 = np.where(cumsum >= 80)[0][0] + 1 if any(cumsum >= 80) else len(cumsum)
    ax3.axvline(idx_80, color='#FDCB6E', linestyle='--')
    ax3.set_xlabel('Number of Clusters (ranked)')
    ax3.set_ylabel('Cumulative % of Cells')
    ax3.set_title(f'C. Cumulative Distribution\n(Top {idx_80} clusters = 80% cells)')
    ax3.legend()
    
    plt.tight_layout()
    plt.savefig('fig2_cluster_analysis.png', dpi=150, bbox_inches='tight', facecolor='white')
    plt.show()
    
    # Summary statistics
    print(f"\nCluster Statistics:")
    print(f"  Total clusters: {len(df_cluster):,}")
    print(f"  Total cells: {cell_counts.sum():,}")
    print(f"  Mean cells/cluster: {cell_counts.mean():,.0f}")
    print(f"  Median cells/cluster: {np.median(cell_counts):,.0f}")
    print(f"  Min/Max: {cell_counts.min():,} / {cell_counts.max():,}")
else:
    print("Cluster data not available for visualization.")

### 4.4 Cell Quality Control Metrics

In [ ]:
if df_cell_metadata is not None:
    fig, axes = plt.subplots(1, 3, figsize=(14, 4))
    fig.suptitle('Cell Quality Control Metrics', fontsize=14, fontweight='bold', y=1.02)
    
    # UMI count distribution
    ax1 = axes[0]
    if 'umi_count' in df_cell_metadata.columns:
        umi = df_cell_metadata['umi_count'].dropna()
        ax1.hist(np.log10(umi + 1), bins=50, color='#74B9FF', edgecolor='white', alpha=0.8)
        ax1.axvline(np.log10(umi.median() + 1), color='#D63031', linestyle='--', 
                   linewidth=2, label=f'Median: {int(umi.median()):,}')
        ax1.set_xlabel('log₁₀(UMI Count + 1)')
        ax1.set_ylabel('Frequency')
        ax1.set_title('A. UMI Count Distribution')
        ax1.legend()
    
    # Doublet score distribution
    ax2 = axes[1]
    if 'doublet_score' in df_cell_metadata.columns:
        doublet = df_cell_metadata['doublet_score'].dropna()
        ax2.hist(doublet, bins=50, color='#A29BFE', edgecolor='white', alpha=0.8)
        ax2.axvline(0.5, color='#D63031', linestyle='--', linewidth=2, label='Threshold: 0.5')
        pct_doublet = (doublet > 0.5).mean() * 100
        ax2.set_xlabel('Doublet Score')
        ax2.set_ylabel('Frequency')
        ax2.set_title(f'B. Doublet Score ({pct_doublet:.1f}% > 0.5)')
        ax2.legend()
    
    # Cells per library
    ax3 = axes[2]
    if 'library_label' in df_cell_metadata.columns:
        lib_counts = df_cell_metadata['library_label'].value_counts()
        ax3.hist(lib_counts.values, bins=30, color='#55EFC4', edgecolor='white', alpha=0.8)
        ax3.axvline(lib_counts.median(), color='#D63031', linestyle='--', 
                   linewidth=2, label=f'Median: {int(lib_counts.median()):,}')
        ax3.set_xlabel('Cells per Library')
        ax3.set_ylabel('Number of Libraries')
        ax3.set_title(f'C. Cells per Library (n={len(lib_counts)})')
        ax3.legend()
    
    plt.tight_layout()
    plt.savefig('fig3_qc_metrics.png', dpi=150, bbox_inches='tight', facecolor='white')
    plt.show()
else:
    print("Cell metadata not available for visualization.")

### 4.5 Taxonomy Hierarchy

In [ ]:
if df_taxonomy is not None and 'cluster_annotation_term_set_name' in df_taxonomy.columns:
    # Count terms per hierarchy level
    hierarchy_counts = df_taxonomy['cluster_annotation_term_set_name'].value_counts()
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    fig.suptitle('Taxonomy Hierarchy Structure', fontsize=14, fontweight='bold', y=1.02)
    
    # Bar chart of terms per level
    colors = plt.cm.Spectral(np.linspace(0.1, 0.9, len(hierarchy_counts)))
    bars = ax.barh(range(len(hierarchy_counts)), hierarchy_counts.values, color=colors)
    ax.set_yticks(range(len(hierarchy_counts)))
    ax.set_yticklabels([h[:35] + '...' if len(h) > 35 else h for h in hierarchy_counts.index], fontsize=9)
    ax.set_xlabel('Number of Terms')
    ax.set_title('Terms per Hierarchy Level')
    ax.invert_yaxis()
    for bar, val in zip(bars, hierarchy_counts.values):
        ax.text(val + 1, bar.get_y() + bar.get_height()/2, str(val), 
                va='center', fontsize=9)
    
    plt.tight_layout()
    plt.savefig('fig4_taxonomy.png', dpi=150, bbox_inches='tight', facecolor='white')
    plt.show()
    
    print(f"\nTaxonomy Summary:")
    print(f"  Total annotation terms: {len(df_taxonomy)}")
    print(f"  Hierarchy levels: {len(hierarchy_counts)}")
else:
    print("Taxonomy data not available for visualization.")

### 4.6 UMAP Embedding Visualization

In [ ]:
# Merge embeddings with cluster assignments for visualization
if df_embeddings is not None and df_cell_cluster is not None:
    # Merge data
    df_viz = df_embeddings.merge(df_cell_cluster, on='cell_label', how='inner')
    
    if df_cell_metadata is not None:
        merge_cols = ['cell_label']
        if 'donor_label' in df_cell_metadata.columns:
            merge_cols.append('donor_label')
        if 'umi_count' in df_cell_metadata.columns:
            merge_cols.append('umi_count')
        if 'doublet_score' in df_cell_metadata.columns:
            merge_cols.append('doublet_score')
        
        df_viz = df_viz.merge(df_cell_metadata[merge_cols], on='cell_label', how='left')
    
    print(f"Visualization data: {len(df_viz):,} cells with embeddings and annotations")
    print(f"Columns: {list(df_viz.columns)}")

In [ ]:
if 'df_viz' in dir() and df_viz is not None and len(df_viz) > 0:
    # Sample for faster plotting
    n_sample = min(50000, len(df_viz))
    df_plot = df_viz.sample(n=n_sample, random_state=42)
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    fig.suptitle('UMAP Embeddings', fontsize=16, fontweight='bold', y=1.01)
    
    # A. Colored by cluster (convert string cluster_alias to numeric codes)
    ax1 = axes[0, 0]
    if 'cluster_alias' in df_plot.columns:
        # Get top 20 clusters and assign numeric codes
        top_clusters = df_plot['cluster_alias'].value_counts().head(20).index.tolist()
        # Create numeric mapping: top clusters get values 0-19, others get -1
        cluster_to_code = {c: i for i, c in enumerate(top_clusters)}
        df_plot['cluster_code'] = df_plot['cluster_alias'].apply(
            lambda x: cluster_to_code.get(x, -1)
        )
        # Plot with numeric codes
        scatter = ax1.scatter(df_plot['x'], df_plot['y'], 
                             c=df_plot['cluster_code'], cmap='tab20',
                             s=1, alpha=0.5, rasterized=True, vmin=-1, vmax=19)
    else:
        ax1.scatter(df_plot['x'], df_plot['y'], s=1, alpha=0.5, c='gray', rasterized=True)
    ax1.set_xlabel('UMAP 1')
    ax1.set_ylabel('UMAP 2')
    ax1.set_title('A. Colored by Cluster (Top 20)')
    ax1.set_aspect('equal')
    
    # B. Colored by donor
    ax2 = axes[0, 1]
    if 'donor_label' in df_plot.columns:
        donors = df_plot['donor_label'].fillna('Unknown')
        donor_codes = pd.Categorical(donors).codes
        scatter = ax2.scatter(df_plot['x'], df_plot['y'], 
                             c=donor_codes, cmap='Spectral',
                             s=1, alpha=0.5, rasterized=True)
        plt.colorbar(scatter, ax=ax2, label='Donor')
    else:
        ax2.scatter(df_plot['x'], df_plot['y'], s=1, alpha=0.5, c='gray', rasterized=True)
    ax2.set_xlabel('UMAP 1')
    ax2.set_ylabel('UMAP 2')
    ax2.set_title('B. Colored by Donor')
    ax2.set_aspect('equal')
    
    # C. Colored by UMI count
    ax3 = axes[1, 0]
    if 'umi_count' in df_plot.columns:
        umi_log = np.log10(df_plot['umi_count'].fillna(1) + 1)
        scatter = ax3.scatter(df_plot['x'], df_plot['y'], 
                             c=umi_log, cmap='viridis',
                             s=1, alpha=0.5, rasterized=True)
        plt.colorbar(scatter, ax=ax3, label='log₁₀(UMI + 1)')
    else:
        ax3.scatter(df_plot['x'], df_plot['y'], s=1, alpha=0.5, c='gray', rasterized=True)
    ax3.set_xlabel('UMAP 1')
    ax3.set_ylabel('UMAP 2')
    ax3.set_title('C. Colored by UMI Count')
    ax3.set_aspect('equal')
    
    # D. Colored by doublet score
    ax4 = axes[1, 1]
    if 'doublet_score' in df_plot.columns:
        doublet = df_plot['doublet_score'].fillna(0)
        scatter = ax4.scatter(df_plot['x'], df_plot['y'], 
                             c=doublet, cmap='RdYlBu_r', vmin=0, vmax=1,
                             s=1, alpha=0.5, rasterized=True)
        plt.colorbar(scatter, ax=ax4, label='Doublet Score')
    else:
        ax4.scatter(df_plot['x'], df_plot['y'], s=1, alpha=0.5, c='gray', rasterized=True)
    ax4.set_xlabel('UMAP 1')
    ax4.set_ylabel('UMAP 2')
    ax4.set_title('D. Colored by Doublet Score')
    ax4.set_aspect('equal')
    
    plt.tight_layout()
    plt.savefig('fig5_umap_embeddings.png', dpi=150, bbox_inches='tight', facecolor='white')
    plt.show()
else:
    print("Embedding data not available for visualization.")

### 4.7 Gene & Library Statistics

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
fig.suptitle('Gene & Library Analysis', fontsize=14, fontweight='bold', y=1.02)

# Gene statistics
ax1 = axes[0]
if df_gene is not None and 'gene_symbol' in df_gene.columns:
    symbol_lengths = df_gene['gene_symbol'].str.len()
    ax1.hist(symbol_lengths, bins=30, color='#81ECEC', edgecolor='white')
    ax1.axvline(symbol_lengths.median(), color='#D63031', linestyle='--', 
               linewidth=2, label=f'Median: {symbol_lengths.median():.0f}')
    ax1.set_xlabel('Gene Symbol Length')
    ax1.set_ylabel('Count')
    ax1.set_title(f'A. Gene Symbol Length (n={len(df_gene):,})')
    ax1.legend()
else:
    ax1.text(0.5, 0.5, 'Gene data not available', ha='center', va='center')
    ax1.set_title('A. Gene Statistics')

# Library statistics
ax2 = axes[1]
if df_library is not None and 'region_of_interest_name' in df_library.columns:
    roi_counts = df_library['region_of_interest_name'].value_counts().head(10)
    colors = plt.cm.Set3(np.linspace(0, 1, len(roi_counts)))
    ax2.barh(range(len(roi_counts)), roi_counts.values, color=colors)
    ax2.set_yticks(range(len(roi_counts)))
    ax2.set_yticklabels([r[:25] for r in roi_counts.index], fontsize=9)
    ax2.set_xlabel('Number of Libraries')
    ax2.set_title(f'B. Libraries by Brain Region (n={len(df_library)})')
    ax2.invert_yaxis()
else:
    ax2.text(0.5, 0.5, 'Library data not available', ha='center', va='center')
    ax2.set_title('B. Library Statistics')

plt.tight_layout()
plt.savefig('fig6_gene_library.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

## 5. Machine Learning Integration

### 5.1 Prepare Data for ML

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

print(f"PyTorch version: {torch.__version__}")
print(f"TensorFlow version: {tf.__version__}")

In [ ]:
# Create synthetic ML data with realistic class imbalance and overlap
np.random.seed(42)

n_cells = 10000
n_genes = 100
n_classes = 15

# Create imbalanced class sizes (power law distribution)
class_probs = np.array([1/(i+1)**0.5 for i in range(n_classes)])
class_probs /= class_probs.sum()
class_sizes = np.random.multinomial(n_cells, class_probs)

X_demo = []
y_demo = []

# Create overlapping clusters with varying separability
class_centers = np.random.randn(n_classes, n_genes) * 3

for i, size in enumerate(class_sizes):
    noise_scale = 0.8 + 0.4 * np.random.rand()
    X_class = class_centers[i] + np.random.randn(size, n_genes) * noise_scale
    
    # Add some mislabeled points (5% noise)
    n_noisy = int(size * 0.05)
    if n_noisy > 0:
        noisy_labels = np.random.choice([j for j in range(n_classes) if j != i], n_noisy)
        y_class = np.array([i] * size)
        y_class[:n_noisy] = noisy_labels
    else:
        y_class = np.array([i] * size)
    
    X_demo.append(X_class)
    y_demo.extend(y_class)

X_demo = np.vstack(X_demo).astype(np.float32)
y_demo = np.array(y_demo)

# Normalize
scaler = StandardScaler()
X_demo = scaler.fit_transform(X_demo)

# Stratified split
X_train, X_test, y_train, y_test = train_test_split(
    X_demo, y_demo, test_size=0.2, stratify=y_demo, random_state=42
)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"Classes: {n_classes}")
print(f"Class distribution (train): {np.bincount(y_train)}")

### 5.2 PyTorch Model

In [ ]:
class CellDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class CellClassifier(nn.Module):
    def __init__(self, n_genes, n_classes, hidden_dims=[256, 128, 64]):
        super().__init__()
        layers = []
        in_dim = n_genes
        for h_dim in hidden_dims:
            layers.extend([
                nn.Linear(in_dim, h_dim),
                nn.BatchNorm1d(h_dim),
                nn.ReLU(),
                nn.Dropout(0.3)
            ])
            in_dim = h_dim
        layers.append(nn.Linear(in_dim, n_classes))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

# Create DataLoaders
train_dataset = CellDataset(X_train, y_train)
test_dataset = CellDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Create model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CellClassifier(n_genes, n_classes).to(device)
print(f"Model on: {device}")

In [ ]:
# Train PyTorch model
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3)

history = {'train_loss': [], 'train_acc': [], 'test_loss': [], 'test_acc': []}

print("Training PyTorch model...")
for epoch in range(25):
    model.train()
    train_loss, train_correct, train_total = 0, 0, 0
    
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_total += y_batch.size(0)
        train_correct += predicted.eq(y_batch).sum().item()
    
    model.eval()
    test_loss, test_correct, test_total = 0, 0, 0
    
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            test_total += y_batch.size(0)
            test_correct += predicted.eq(y_batch).sum().item()
    
    train_acc = 100. * train_correct / train_total
    test_acc = 100. * test_correct / test_total
    history['train_loss'].append(train_loss / len(train_loader))
    history['train_acc'].append(train_acc)
    history['test_loss'].append(test_loss / len(test_loader))
    history['test_acc'].append(test_acc)
    
    scheduler.step(test_loss)
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/25: Train Acc: {train_acc:.1f}% | Test Acc: {test_acc:.1f}%")

print(f"\nFinal PyTorch Test Accuracy: {history['test_acc'][-1]:.1f}%")

### 5.3 TensorFlow Model

In [ ]:
# Build TensorFlow model
tf_model = keras.Sequential([
    keras.layers.Input(shape=(n_genes,)),
    keras.layers.Dense(256),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(128),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(n_classes, activation='softmax')
])

tf_model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Training TensorFlow model...")
tf_history = tf_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=25,
    batch_size=128,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(patience=3),
        keras.callbacks.EarlyStopping(patience=7, restore_best_weights=True)
    ]
)

test_loss, test_acc = tf_model.evaluate(X_test, y_test, verbose=0)
print(f"Final TensorFlow Test Accuracy: {test_acc*100:.1f}%")

### 5.4 Training Curves Comparison

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
fig.suptitle('Training Comparison: PyTorch vs TensorFlow', fontsize=14, fontweight='bold')

ax1 = axes[0]
ax1.plot(history['train_loss'], 'b-', label='PyTorch Train', linewidth=2)
ax1.plot(history['test_loss'], 'b--', label='PyTorch Test', linewidth=2)
ax1.plot(tf_history.history['loss'], 'r-', label='TensorFlow Train', linewidth=2)
ax1.plot(tf_history.history['val_loss'], 'r--', label='TensorFlow Val', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2 = axes[1]
ax2.plot(history['train_acc'], 'b-', label='PyTorch Train', linewidth=2)
ax2.plot(history['test_acc'], 'b--', label='PyTorch Test', linewidth=2)
ax2.plot([x*100 for x in tf_history.history['accuracy']], 'r-', label='TensorFlow Train', linewidth=2)
ax2.plot([x*100 for x in tf_history.history['val_accuracy']], 'r--', label='TensorFlow Val', linewidth=2)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Training Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('fig7_training_curves.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

### 5.5 Model Behavior Analysis

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.manifold import TSNE

# Get predictions from both models
model.eval()
with torch.no_grad():
    X_test_tensor = torch.FloatTensor(X_test).to(device)
    pytorch_logits = model(X_test_tensor).cpu().numpy()
    pytorch_preds = pytorch_logits.argmax(axis=1)
    pytorch_probs = torch.softmax(torch.FloatTensor(pytorch_logits), dim=1).numpy()

tf_probs = tf_model.predict(X_test, verbose=0)
tf_preds = tf_probs.argmax(axis=1)

# Calculate metrics
pytorch_conf = pytorch_probs.max(axis=1)
tf_conf = tf_probs.max(axis=1)
pytorch_correct = pytorch_preds == y_test
tf_correct = tf_preds == y_test

print("Prediction Summary:")
print(f"  PyTorch accuracy: {pytorch_correct.mean()*100:.1f}%")
print(f"  TensorFlow accuracy: {tf_correct.mean()*100:.1f}%")
print(f"  Agreement: {(pytorch_preds == tf_preds).mean()*100:.1f}%")

In [ ]:
fig = plt.figure(figsize=(16, 12))
fig.suptitle('Model Behavior Analysis', fontsize=16, fontweight='bold', y=0.98)

# A. Confidence Distribution - Correct vs Incorrect (PyTorch)
ax1 = fig.add_subplot(2, 3, 1)
bins = np.linspace(0, 1, 31)
ax1.hist(pytorch_conf[pytorch_correct], bins=bins, alpha=0.7, label='Correct', color='#27AE60', density=True)
ax1.hist(pytorch_conf[~pytorch_correct], bins=bins, alpha=0.7, label='Incorrect', color='#E74C3C', density=True)
ax1.axvline(pytorch_conf[pytorch_correct].mean(), color='#27AE60', linestyle='--', linewidth=2)
ax1.axvline(pytorch_conf[~pytorch_correct].mean(), color='#E74C3C', linestyle='--', linewidth=2)
ax1.set_xlabel('Prediction Confidence')
ax1.set_ylabel('Density')
ax1.set_title('A. PyTorch Confidence Distribution')
ax1.legend()

# B. Confidence Distribution (TensorFlow)
ax2 = fig.add_subplot(2, 3, 2)
ax2.hist(tf_conf[tf_correct], bins=bins, alpha=0.7, label='Correct', color='#27AE60', density=True)
ax2.hist(tf_conf[~tf_correct], bins=bins, alpha=0.7, label='Incorrect', color='#E74C3C', density=True)
ax2.axvline(tf_conf[tf_correct].mean(), color='#27AE60', linestyle='--', linewidth=2)
ax2.axvline(tf_conf[~tf_correct].mean(), color='#E74C3C', linestyle='--', linewidth=2)
ax2.set_xlabel('Prediction Confidence')
ax2.set_ylabel('Density')
ax2.set_title('B. TensorFlow Confidence Distribution')
ax2.legend()

# C. Per-Class Accuracy Comparison
ax3 = fig.add_subplot(2, 3, 3)
class_acc_pt = []
class_acc_tf = []
for c in range(n_classes):
    mask = y_test == c
    class_acc_pt.append((pytorch_preds[mask] == y_test[mask]).mean() * 100 if mask.sum() > 0 else 0)
    class_acc_tf.append((tf_preds[mask] == y_test[mask]).mean() * 100 if mask.sum() > 0 else 0)

x = np.arange(n_classes)
width = 0.35
ax3.bar(x - width/2, class_acc_pt, width, label='PyTorch', color='#3498DB', alpha=0.8)
ax3.bar(x + width/2, class_acc_tf, width, label='TensorFlow', color='#E67E22', alpha=0.8)
ax3.axhline(np.mean(class_acc_pt), color='#3498DB', linestyle='--', alpha=0.5)
ax3.axhline(np.mean(class_acc_tf), color='#E67E22', linestyle='--', alpha=0.5)
ax3.set_xlabel('Class ID')
ax3.set_ylabel('Accuracy (%)')
ax3.set_title('C. Per-Class Accuracy')
ax3.legend()
ax3.set_ylim(0, 105)

# D. Confusion Matrix (PyTorch)
ax4 = fig.add_subplot(2, 3, 4)
cm = confusion_matrix(y_test, pytorch_preds)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
im = ax4.imshow(cm_norm, cmap='Blues', vmin=0, vmax=1)
ax4.set_xlabel('Predicted')
ax4.set_ylabel('True')
ax4.set_title('D. Confusion Matrix (PyTorch)')
plt.colorbar(im, ax=ax4, label='Proportion')

# E. t-SNE of Learned Embeddings
ax5 = fig.add_subplot(2, 3, 5)

class EmbeddingExtractor(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.features = nn.Sequential(*list(model.network.children())[:-1])
    def forward(self, x):
        return self.features(x)

extractor = EmbeddingExtractor(model).to(device)
extractor.eval()

n_tsne = min(2000, len(X_test))
idx = np.random.choice(len(X_test), n_tsne, replace=False)
with torch.no_grad():
    embeddings = extractor(torch.FloatTensor(X_test[idx]).to(device)).cpu().numpy()

tsne = TSNE(n_components=2, perplexity=30, random_state=42, max_iter=500)
emb_2d = tsne.fit_transform(embeddings)

scatter = ax5.scatter(emb_2d[:, 0], emb_2d[:, 1], c=y_test[idx], cmap='tab20', s=10, alpha=0.6)
ax5.set_xlabel('t-SNE 1')
ax5.set_ylabel('t-SNE 2')
ax5.set_title('E. Learned Embeddings (t-SNE)')

# F. Model Agreement Analysis
ax6 = fig.add_subplot(2, 3, 6)
both_correct = (pytorch_correct & tf_correct).sum()
both_wrong = (~pytorch_correct & ~tf_correct).sum()
pt_only = (pytorch_correct & ~tf_correct).sum()
tf_only = (~pytorch_correct & tf_correct).sum()

categories = ['Both Correct', 'Both Wrong', 'PyTorch Only', 'TF Only']
counts = [both_correct, both_wrong, pt_only, tf_only]
colors = ['#27AE60', '#E74C3C', '#3498DB', '#E67E22']

non_zero = [(c, cat, col) for c, cat, col in zip(counts, categories, colors) if c > 0]
if len(non_zero) > 0:
    counts_nz = [x[0] for x in non_zero]
    labels_nz = [x[1] for x in non_zero]
    colors_nz = [x[2] for x in non_zero]
    wedges, texts, autotexts = ax6.pie(counts_nz, labels=labels_nz, autopct='%1.1f%%',
                                        colors=colors_nz, startangle=90,
                                        wedgeprops=dict(edgecolor='white', linewidth=2))
ax6.set_title('F. Model Agreement Analysis')

plt.tight_layout()
plt.savefig('fig8_model_behavior.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

print("\n" + "="*60)
print("DETAILED MODEL BEHAVIOR STATISTICS")
print("="*60)
print(f"\nConfidence Analysis:")
print(f"  PyTorch - Correct: mean={pytorch_conf[pytorch_correct].mean():.3f}")
print(f"  PyTorch - Wrong:   mean={pytorch_conf[~pytorch_correct].mean():.3f}")
print(f"  TensorFlow - Correct: mean={tf_conf[tf_correct].mean():.3f}")
print(f"  TensorFlow - Wrong:   mean={tf_conf[~tf_correct].mean():.3f}")
print(f"\nModel Agreement:")
print(f"  Both correct: {both_correct} ({both_correct/len(y_test)*100:.1f}%)")
print(f"  Both wrong:   {both_wrong} ({both_wrong/len(y_test)*100:.1f}%)")
print(f"  PyTorch only: {pt_only} ({pt_only/len(y_test)*100:.1f}%)")
print(f"  TF only:      {tf_only} ({tf_only/len(y_test)*100:.1f}%)")

## 6. Summary

This notebook demonstrated comprehensive analysis of the HMBA Basal Ganglia dataset:

### EDA Highlights
- Multi-species dataset with diverse donors
- Hierarchical cell type taxonomy with multiple annotation levels
- Quality-controlled cells with UMAP embeddings
- Rich metadata across 14 CSV files loaded via Croissant RecordSets

### ML Results
- PyTorch and TensorFlow models trained successfully
- Model behavior analysis shows confidence calibration
- t-SNE visualization reveals learned representations

In [ ]:
print("="*60)
print("ANALYSIS COMPLETE")
print("="*60)
print(f"\nDataset: {metadata['name']}")
print(f"Total files: {len(distributions)}")
print(f"  - H5AD: {len(h5ad_files)}")
print(f"  - Parquet: {len(parquet_files)}")
print(f"  - CSV: {len(csv_files)}")
print(f"\nFigures generated: 8")
print(f"\nFinal Model Accuracies:")
print(f"  PyTorch: {history['test_acc'][-1]:.1f}%")
print(f"  TensorFlow: {tf_history.history['val_accuracy'][-1]*100:.1f}%")